In [ ]:
"""
Credit Risk service:
- Loads model if artifacts exist, else uses a simple heuristic
- Returns PD (probability of default) and feature attributions (mock SHAP)
"""
import os
import pickle
import numpy as np

ARTIFACT_PATH = os.getenv("CREDIT_MODEL_PATH", "/models/credit_model.pkl")

def _heuristic_pd(ltv: float, dti: float, delinq: int, ead: float):
    score = 0.4*ltv + 0.4*dti + 0.1*delinq + 0.1*np.log1p(max(ead, 0.0))
    pd = 1 / (1 + np.exp(-(score - 1.5)))  # squash
    return float(np.clip(pd, 0.01, 0.99))

def predict_pd(features: dict):
    if os.path.exists(ARTIFACT_PATH):
        try:
            with open(ARTIFACT_PATH, "rb") as f:
                model = pickle.load(f)
            X = np.array([[features["ltv"], features["dti"],
                           features["delinquency_history"],
                           features["exposure_at_default"]]])
            proba = model.predict_proba(X)[0,1]
            pd = float(np.clip(proba, 0.01, 0.99))
        except Exception:
            pd = _heuristic_pd(features["ltv"], features["dti"],
                               features["delinquency_history"],
                               features["exposure_at_default"])
    else:
        pd = _heuristic_pd(features["ltv"], features["dti"],
                           features["delinquency_history"],
                           features["exposure_at_default"])

    total = features["ltv"] + features["dti"] + features["delinquency_history"] + features["exposure_at_default"]
    if total == 0: total = 1.0
    attributions = {
        "ltv": round(features["ltv"]/total, 3),
        "dti": round(features["dti"]/total, 3),
        "delinquency_history": round(features["delinquency_history"]/total, 3),
        "exposure_at_default": round(features["exposure_at_default"]/total, 3),
    }
    return {"pd": pd, "attributions": attributions}